In [146]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

# 进销口分析

In [2]:
input1 = pd.read_excel('附件1.xlsx',sheet_name='进项发票信息')
output1 = pd.read_excel('附件1.xlsx',sheet_name='销项发票信息')

In [22]:
for num in range(1,124):
    id1 = 'E'+str(num)
    input1[input1['企业代号']==id1]

In [41]:
num = 1
id1 = 'E'+str(num)
tem = input1[input1['企业代号']==id1]
all_num = tem.shape[0]
fei_num = tem[tem['发票状态']=='作废发票'].shape[0]
ratio = fei_num/all_num
ratio

0.05579773321708806

In [47]:
r_in=[]

for num in range(1,124):
    id1 = 'E'+str(num)
    tem = input1[input1['企业代号']==id1]
    all_num = tem.shape[0]
    fei_num = tem[tem['发票状态']=='作废发票'].shape[0]
    ratio = fei_num/all_num
    
    r_in.append(ratio)
    
r_out=[]

for num in range(1,124):
    id1 = 'E'+str(num)
    tem = output1[output1['企业代号']==id1]
    all_num = tem.shape[0]
    fei_num = tem[tem['发票状态']=='作废发票'].shape[0]
    ratio = fei_num/all_num
    
    r_out.append(ratio)

    

name=[]    
for num in range(1,124):
    id1 = 'E'+str(num)
    name.append(id1)
    

In [55]:
in_num=[]
for num in range(1,124):
    id1 = 'E'+str(num)
    tem = input1[input1['企业代号']==id1]
    all_num = tem.shape[0]
    
    in_num.append(all_num)
    
out_num=[]
for num in range(1,124):
    id1 = 'E'+str(num)
    tem = output1[output1['企业代号']==id1]
    all_num = tem.shape[0]
    
    out_num.append(all_num)

In [56]:
data = {'企业名称':name,
    '进项发票的作废比例':r_in   ,
      '销项发票的作废比例':r_out  ,
        '进项发票数':in_num ,
        '销项发票数':out_num
}

df=pd.DataFrame(data=data)
df

,企业名称,进项发票的作废比例,销项发票的作废比例,进项发票数,销项发票数
0,E1,0.055798,0.027620,3441,8110
1,E2,0.022422,0.082002,32156,12707
2,E3,0.042535,0.015993,4561,24073
3,E4,0.066308,0.085164,558,2231
4,E5,0.039189,0.051887,2169,1060
...,...,...,...,...,...
118,E119,0.003175,0.142857,315,21
119,E120,0.027778,0.689655,36,29
120,E121,0.000000,0.123656,50,186
121,E122,0.020833,0.135593,48,118


In [57]:
df.to_csv('附件1作废发票比例.csv',encoding='gbk')

# 企业文字提取

In [59]:
data = pd.read_excel('附件1.xlsx',sheet_name='企业信息')
data

,企业代号,企业名称,信誉评级,是否违约
0,E1,***电器销售有限公司,A,否
1,E2,***技术有限责任公司,A,否
2,E3,***电子(中国)有限公司***分公司,C,否
3,E4,***发展有限责任公司,C,否
4,E5,***供应链管理有限公司,B,否
...,...,...,...,...
118,E119,***药房,D,是
119,E120,***陈列广告有限公司,D,是
120,E121,***药业连锁有限公司***药店,D,是
121,E122,***商贸有限责任公司,D,是


# 时间序列分析

In [67]:
for i in range(len(input1)):         # 删除作废发票
    if input1.loc[i,'发票状态']=='作废发票':
        input1.drop(i,inplace=True)

In [68]:
for i in range(len(output1)):
    if output1.loc[i,'发票状态']=='作废发票':
        output1.drop(i,inplace=True)

In [72]:
input1.reset_index(drop = True,inplace=True)
output1.reset_index(drop = True,inplace=True)

In [73]:
input1['开票日期'] = pd.to_datetime(input1['开票日期'])
output1['开票日期'] = pd.to_datetime(output1['开票日期'])

In [219]:
deal1=[]
deal2=[]
deal3=[]
deal4=[]
deal5=[]
for num in range(1,124):      # 遍历目标企业
    l_1 = 0
    l_2 = 0
    l_3 = 0       # 一年、两年、三年、四年的交易方
    l_4 = 0
    l_5 = 0

    id1 = 'E'+str(num)
    a = input1[input1['企业代号']==id1]
    a = a.reset_index(drop=True)
    other = np.unique(a['销方单位代号'])    # 交易方代号的唯一值
    
    year = a['开票日期'][len(a)-1].year - a['开票日期'][0].year+1  
    begin = a['开票日期'][0].year
    
    for i in range(len(other)):       # 遍历交易企业
        cou=0
        company = other[i]      # 选中的交易企业
        for j in range(year):    # 遍历每一年
            tem = a[(a['开票日期'] >str(begin+j) )& (a['开票日期'] <str(begin+j+1))]['销方单位代号'].unique()  # 一年内的交易企业
            if company in tem:
                cou +=1
                
        if cou == 1:
            l_1+=1
        elif cou == 2:
            l_2+=1
        elif cou ==3:
            l_3+= 1
        elif cou==4:
            l_4+=1
        elif cou == 5:
            l_5+=1
                
                
    deal1.append(l_1)
    deal2.append(l_2)
    deal3.append(l_3)        
    deal4.append(l_4)        
    deal5.append(l_5)

In [222]:
da1 = {'持续一年期的交易企业个数':deal1,
     '持续二年期的交易企业个数': deal2,
     '持续三年期的交易企业个数': deal3,
     '持续四年前的交易企业个数':deal4,
       '持续五年期的交易企业个数':deal5
     }

In [223]:
jx = pd.DataFrame(data=da1)
jx

,持续一年期的交易企业个数,持续二年期的交易企业个数,持续三年期的交易企业个数,持续四年前的交易企业个数,持续五年期的交易企业个数
0,332,74,27,3,0
1,2424,780,314,103,0
2,386,103,58,23,0
3,94,21,13,2,0
4,324,47,19,2,0
...,...,...,...,...,...
118,7,6,2,2,0
119,13,3,0,0,0
120,7,2,2,0,0
121,18,3,1,0,0


In [301]:
jx.to_csv('上游持续交易企业数.csv',encoding='gbk')

In [230]:
deal1=[]
deal2=[]
deal3=[]
deal4=[]
for num in range(1,124):      # 遍历目标企业
    l_1 = 0
    l_2 = 0
    l_3 = 0       # 一年、两年、三年、四年的交易方
    l_4 = 0

    id1 = 'E'+str(num)
    a = output1[output1['企业代号']==id1]
    a = a.reset_index(drop=True)
    other = np.unique(a['购方单位代号'])    # 交易方代号的唯一值
    
    year = a['开票日期'][len(a)-1].year - a['开票日期'][0].year+1  
    begin = a['开票日期'][0].year
    
    for i in range(len(other)):       # 遍历交易企业
        cou=0
        company = other[i]      # 选中的交易企业
        for j in range(year):    # 遍历每一年
            tem = a[(a['开票日期'] >str(begin+j) )& (a['开票日期'] <str(begin+j+1))]['购方单位代号'].unique()  # 一年内的交易企业
            if company in tem:
                cou +=1
                
        if cou == 1:
            l_1+=1
        elif cou == 2:
            l_2+=1
        elif cou ==3:
            l_3+= 1
        elif cou==4:
            l_4+=1

                
                
    deal1.append(l_1)
    deal2.append(l_2)
    deal3.append(l_3)        
    deal4.append(l_4)        


In [231]:
da2= {'持续一年期的交易企业个数':deal1,
     '持续二年期的交易企业个数': deal2,
     '持续三年期的交易企业个数': deal3,
     '持续四年前的交易企业个数':deal4
     }

xx = pd.DataFrame(data=da2)
xx 

,持续一年期的交易企业个数,持续二年期的交易企业个数,持续三年期的交易企业个数,持续四年前的交易企业个数
0,179,51,51,71
1,1147,268,121,43
2,48,48,25,14
3,16,10,0,0
4,16,15,2,0
...,...,...,...,...
118,12,1,1,0
119,6,0,0,0
120,110,6,0,0
121,67,2,0,0


In [300]:
xx.to_csv('下游持续交易企业.csv',encoding='gbk')

In [233]:
in_y=[]
for num in range(1,124):      # 遍历目标企业
    l_1 = 0
    l_2 = 0
    l_3 = 0       # 一年、两年、三年、四年的交易方
    l_4 = 0
    l_5 = 0

    id1 = 'E'+str(num)
    a = input1[input1['企业代号']==id1]
    a = a.reset_index(drop=True)
    other = np.unique(a['销方单位代号'])    # 交易方代号的唯一值
    
    year = a['开票日期'][len(a)-1].year - a['开票日期'][0].year+1  
    in_y.append(year)



out_y=[]
for num in range(1,124):      # 遍历目标企业
    l_1 = 0
    l_2 = 0
    l_3 = 0       # 一年、两年、三年、四年的交易方
    l_4 = 0

    id1 = 'E'+str(num)
    a = output1[output1['企业代号']==id1]
    a = a.reset_index(drop=True)
    other = np.unique(a['购方单位代号'])    # 交易方代号的唯一值
    
    year = a['开票日期'][len(a)-1].year - a['开票日期'][0].year+1  
    out_y.append(year)

In [245]:
new_in = input1[(input1['开票日期']<'2020')]    # 2020年之前的进项数据
new_out = output1[(output1['开票日期']<'2020')]

In [246]:
new_in

,企业代号,发票号码,开票日期,销方单位代号,金额,税额,价税合计,发票状态
0,E1,3390939,2017-07-18,A00297,-943.40,-56.60,-1000.00,有效发票
1,E1,3390940,2017-07-18,A00297,-4780.24,-286.81,-5067.05,有效发票
2,E1,3390941,2017-07-18,A00297,943.40,56.60,1000.00,有效发票
3,E1,3390942,2017-07-18,A00297,4780.24,286.81,5067.05,有效发票
4,E1,9902669,2017-08-07,A05061,326.21,9.79,336.00,有效发票
...,...,...,...,...,...,...,...,...
203333,E122,42055639,2019-03-25,A13332,17636.09,1058.17,18694.26,有效发票
203334,E122,54706234,2019-04-17,A08967,223.30,6.70,230.00,有效发票
203336,E123,38493295,2017-12-15,A03624,264.15,15.85,280.00,有效发票
203337,E123,95472001,2018-12-29,A03626,264.15,15.85,280.00,有效发票


In [292]:
ur = []

dd = []
for num in range(1,124):      # 遍历目标企业
    
    id1 = 'E'+str(num)
    a = new_in[new_in['企业代号']==id1]
    a = a.reset_index(drop=True)
    
    end_1 = a['开票日期'][len(a)-1].year
    begin_1 = a['开票日期'][0].year
    
    begin_ji_1 = a[(a['开票日期']>str(begin_1))&(a['开票日期']<str(begin_1+1))]['金额'].sum()
    end_ji_1 =  a[(a['开票日期']>str(end_1))&(a['开票日期']<str(end_1+1))]['金额'].sum()
    
    
    b = new_out[new_out['企业代号']==id1]
    b = b.reset_index(drop=True)
    
    end_2 = b['开票日期'][len(b)-1].year
    begin_2 = b['开票日期'][0].year
    
    begin_ji_2 = b[(b['开票日期']>str(begin_2))&(b['开票日期']<str(begin_2+1))]['金额'].sum()
    end_ji_2 =  b[(b['开票日期']>str(end_2))&(b['开票日期']<str(end_2+1))]['金额'].sum()
    
    up_rate = (end_ji_2-end_ji_1)/(begin_ji_2-begin_ji_1)
    
    d = (end_ji_2-end_ji_1)-(begin_ji_2-begin_ji_1)
    
    
    ur.append(up_rate)
    dd.append(d)

In [293]:
dd

[-193872135.96000051,
 110302536.47999997,
 -115477286.08000003,
 -1168509293.45,
 4068217.1499999883,
 14520912.319999982,
 -170352223.81,
 10175963.700000003,
 8701373.75999999,
 82858784.61000001,
 -38346246.26000001,
 -84557485.89000002,
 17290905.08,
 8017782.599999994,
 28666792.4,
 39872630.50000001,
 10825143.39,
 17393819.630000006,
 457386.0400000019,
 4408596.540000007,
 9020452.639999993,
 34495523.8,
 -6130097.840000002,
 21065382.97,
 3950781.0299999993,
 801968.4499999988,
 340587.8900000041,
 19643668.89,
 11138949.26,
 2738199.3100000005,
 7131326.480000002,
 11818843.080000004,
 -1247606.5799999963,
 -1442409.789999997,
 4954392.8100000005,
 5859723.15,
 20100.5100000035,
 3241096.44999999,
 13935775.469999997,
 13206723.029999997,
 12254350.030000005,
 12910420.96,
 1390239.3199999994,
 899638.66,
 -697757.9200000023,
 1283506.089999996,
 1285307.1699999971,
 17195806.14,
 -4707428.380000005,
 6781297.75,
 5553169.2200000025,
 1095999.810000001,
 244683.3399999952,
 

In [294]:
data3 = {'绝对数变化':dd,
        '比例变化':ur}
df3 = pd.DataFrame(data=data3)
df3

,绝对数变化,比例变化
0,-1.938721e+08,1.720614
1,1.103025e+08,2.197674
2,-1.154773e+08,0.516353
3,-1.168509e+09,0.053642
4,4.068217e+06,85.227087
...,...,...
118,-6.861922e+04,6.999117
119,8.830950e+03,1.139033
120,6.906522e+05,-0.075122
121,-2.734729e+04,-0.681496


In [297]:
for i in range(len(df3)):
    if (df3.loc[i,'绝对数变化']>0) & (df3.loc[i,'比例变化']<0):     # 说明扭亏为盈
        df3.loc[i,'是否扭亏为盈利']=1
    else:
        df3.loc[i,'是否扭亏为盈利']=0

for i in range(len(df3)):
    if (df3.loc[i,'绝对数变化']<0) & (df3.loc[i,'比例变化']<0):     # 说明扭亏为盈
        df3.loc[i,'是否变为亏损']=1
    else:
        df3.loc[i,'是否变为亏损']=0        

df3

,绝对数变化,比例变化,是否扭亏为盈利,是否变为亏损
0,-1.938721e+08,1.720614,0.0,0.0
1,1.103025e+08,2.197674,0.0,0.0
2,-1.154773e+08,0.516353,0.0,0.0
3,-1.168509e+09,0.053642,0.0,0.0
4,4.068217e+06,85.227087,0.0,0.0
...,...,...,...,...
118,-6.861922e+04,6.999117,0.0,0.0
119,8.830950e+03,1.139033,0.0,0.0
120,6.906522e+05,-0.075122,1.0,0.0
121,-2.734729e+04,-0.681496,0.0,1.0


In [299]:
df3.to_csv('发展程度.csv',encoding='gbk')

In [290]:
num=5
id1 = 'E'+str(num)
a = new_in[new_in['企业代号']==id1]
a = a.reset_index(drop=True)
    
end_1 = a['开票日期'][len(a)-1].year
begin_1 = a['开票日期'][0].year
    
begin_ji_1 = a[(a['开票日期']>str(begin_1))&(a['开票日期']<str(begin_1+1))]['金额'].sum()
end_ji_1 =  a[(a['开票日期']>str(end_1))&(a['开票日期']<str(end_1+1))]['金额'].sum()
    
    
b = new_out[new_out['企业代号']==id1]
b = b.reset_index(drop=True)
    
end_2 = b['开票日期'][len(b)-1].year
begin_2 = b['开票日期'][0].year
    
begin_ji_2 = b[(b['开票日期']>str(begin_2))&(b['开票日期']<str(begin_2+1))]['金额'].sum()
end_ji_2 =  b[(b['开票日期']>str(end_2))&(b['开票日期']<str(end_2+1))]['金额'].sum()

up_rate = (end_ji_2-end_ji_1)/(begin_ji_2-begin_ji_1)

In [291]:
print(end_ji_2-end_ji_1)
print(begin_ji_2-begin_ji_1)

4116517.7299999893
48300.580000001006


In [289]:
if 

[1.7206138607491175,
 2.1976741755350835,
 0.5163533480678486,
 0.053642120805519077,
 85.22708692110744,
 6.220033757117115,
 0.28299993853695593,
 1.1903434951584118,
 1.1026416281740918,
 5.6952388287385975,
 0.06224696926670617,
 0.3142310613924136,
 1.6246590416677933,
 1.2608609677500382,
 2.030952353418196,
 4.42752647859875,
 4.6091111208456725,
 25.37000437751128,
 1.2952080964666253,
 3.448482477010922,
 -107.85542263470977,
 -68.36411555912315,
 -3.1526842431065614,
 3.7508149740262886,
 1.8719903885132894,
 -1.5351713542943943,
 -4.576462137815929,
 7.404078615947699,
 1.8563019203053552,
 1.2348027347399602,
 1.6791198336803776,
 2.0636008405295763,
 0.5125450991083275,
 140.72763660986513,
 -11.283839443261392,
 3.21411258457861,
 0.9453293654883062,
 1.3313401805724063,
 7.452725580324382,
 6.115626835894702,
 4.989140651177328,
 12.135349485779685,
 1.1820756853858494,
 6.562504223791024,
 -5.818097757171771,
 12.069900028107975,
 2.966145392466539,
 5.250937640696781,


# 第二问发票数据处理

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

input1 = pd.read_excel('附件2.xlsx',sheet_name='进项发票信息')
output1 = pd.read_excel('附件2.xlsx',sheet_name='销项发票信息')

In [7]:
# r_in=[]

# for num in range(1,303):
#     id1 = 'E'+str(num+123)
#     tem = input1[input1['企业代号']==id1]
#     all_num = tem.shape[0]
#     fei_num = tem[tem['发票状态']=='作废发票'].shape[0]
#     ratio = fei_num/all_num
    
#     r_in.append(ratio)
    
r_out=[]

for num in range(1,303):
    id1 = 'E'+str(num+123)
    tem = output1[output1['企业代号']==id1]
    all_num = tem.shape[0]
    fei_num = tem[tem['发票状态']=='作废发票'].shape[0]
    ratio = fei_num/all_num
    
    r_out.append(ratio)

    

name=[]    
for num in range(1,303):
    id1 = 'E'+str(num+123)
    name.append(id1)
    

In [8]:
in_num=[]
for num in range(1,303):
    id1 = 'E'+str(num+123)
    tem = input1[input1['企业代号']==id1]
    all_num = tem.shape[0]
    
    in_num.append(all_num)
    
out_num=[]
for num in range(1,303):
    id1 = 'E'+str(num+123)
    tem = output1[output1['企业代号']==id1]
    all_num = tem.shape[0]
    
    out_num.append(all_num)

In [10]:
data = {'企业名称':name,
    '进项发票的作废比例':r_in   ,
      '销项发票的作废比例':r_out  
}

df=pd.DataFrame(data=data)
df

,企业名称,进项发票的作废比例,销项发票的作废比例
0,E124,0.123313,0.150039
1,E125,0.123817,0.134796
2,E126,0.033771,0.166227
3,E127,0.017931,0.030303
4,E128,0.027312,0.074900
...,...,...,...
297,E421,0.000000,0.034483
298,E422,0.000000,0.100000
299,E423,0.000000,0.142857
300,E424,0.000000,0.139535


In [11]:
df.to_csv('第二问作废比例.csv',encoding='gbk')

In [12]:
new_in = input1[(input1['开票日期']<'2020')]    # 2020年之前的进项数据
new_out = output1[(output1['开票日期']<'2020')]

In [13]:
ur = []

dd = []
for num in range(1,303):      # 遍历目标企业
    
    id1 = 'E'+str(num+123)
    a = new_in[new_in['企业代号']==id1]
    a = a.reset_index(drop=True)
    
    end_1 = a['开票日期'][len(a)-1].year
    begin_1 = a['开票日期'][0].year
    
    begin_ji_1 = a[(a['开票日期']>str(begin_1))&(a['开票日期']<str(begin_1+1))]['金额'].sum()
    end_ji_1 =  a[(a['开票日期']>str(end_1))&(a['开票日期']<str(end_1+1))]['金额'].sum()
    
    
    b = new_out[new_out['企业代号']==id1]
    b = b.reset_index(drop=True)
    
    end_2 = b['开票日期'][len(b)-1].year
    begin_2 = b['开票日期'][0].year
    
    begin_ji_2 = b[(b['开票日期']>str(begin_2))&(b['开票日期']<str(begin_2+1))]['金额'].sum()
    end_ji_2 =  b[(b['开票日期']>str(end_2))&(b['开票日期']<str(end_2+1))]['金额'].sum()
    
    up_rate = (end_ji_2-end_ji_1)/(begin_ji_2-begin_ji_1)
    
    d = (end_ji_2-end_ji_1)-(begin_ji_2-begin_ji_1)
    
    
    ur.append(up_rate)
    dd.append(d)

In [14]:
data3 = {'绝对数变化':dd,
        '比例变化':ur}
df3 = pd.DataFrame(data=data3)
df3

,绝对数变化,比例变化
0,-1.065813e+08,-2.086819
1,-1.697853e+08,-1.069323
2,1.540513e+08,2.527125
3,-8.365728e+07,0.677000
4,8.388073e+07,2.918189
...,...,...
297,-5.554860e+04,0.110407
298,1.892226e+04,6.340986
299,1.143155e+04,0.214670
300,-8.175862e+04,2.007337


In [15]:
for i in range(len(df3)):
    if (df3.loc[i,'绝对数变化']>0) & (df3.loc[i,'比例变化']<0):     # 说明扭亏为盈
        df3.loc[i,'是否扭亏为盈利']=1
    else:
        df3.loc[i,'是否扭亏为盈利']=0

for i in range(len(df3)):
    if (df3.loc[i,'绝对数变化']<0) & (df3.loc[i,'比例变化']<0):     # 说明扭亏为盈
        df3.loc[i,'是否变为亏损']=1
    else:
        df3.loc[i,'是否变为亏损']=0        

df3

,绝对数变化,比例变化,是否扭亏为盈利,是否变为亏损
0,-1.065813e+08,-2.086819,0.0,1.0
1,-1.697853e+08,-1.069323,0.0,1.0
2,1.540513e+08,2.527125,0.0,0.0
3,-8.365728e+07,0.677000,0.0,0.0
4,8.388073e+07,2.918189,0.0,0.0
...,...,...,...,...
297,-5.554860e+04,0.110407,0.0,0.0
298,1.892226e+04,6.340986,0.0,0.0
299,1.143155e+04,0.214670,0.0,0.0
300,-8.175862e+04,2.007337,0.0,0.0


In [16]:
df3.to_csv('第二问绝对数和比例变化.csv',encoding='gbk')